In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
air = pd.read_csv('dataset/airpassenger.csv')

In [5]:
air.head()

,DATAFLOW,LAST UPDATE,freq,unit,tra_meas,tra_cov,schedule,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:AVIA_PAOC(1.0),15/05/25 11:00:00,Monthly,Passenger,Passengers on board,Total transport,Total,Austria,2002-01,935901,NaN,NaN
1,ESTAT:AVIA_PAOC(1.0),15/05/25 11:00:00,Monthly,Passenger,Passengers on board,Total transport,Total,Austria,2002-02,966771,NaN,NaN
2,ESTAT:AVIA_PAOC(1.0),15/05/25 11:00:00,Monthly,Passenger,Passengers on board,Total transport,Total,Austria,2002-03,1184268,NaN,NaN
3,ESTAT:AVIA_PAOC(1.0),15/05/25 11:00:00,Monthly,Passenger,Passengers on board,Total transport,Total,Austria,2002-04,1143982,NaN,NaN
4,ESTAT:AVIA_PAOC(1.0),15/05/25 11:00:00,Monthly,Passenger,Passengers on board,Total transport,Total,Austria,2002-05,1326116,NaN,NaN


In [7]:
air['TIME_PERIOD'] = pd.to_datetime(air['TIME_PERIOD'])

In [9]:
air = air.set_index(air['TIME_PERIOD'])

In [10]:
air_germany = air[air['geo'] == 'Germany']

In [13]:
air_germany.index

DatetimeIndex(['2002-01-01', '2002-02-01', '2002-03-01', '2002-04-01',
               '2002-05-01', '2002-06-01', '2002-07-01', '2002-08-01',
               '2002-09-01', '2002-10-01',
               ...
               '2024-04-01', '2024-05-01', '2024-06-01', '2024-07-01',
               '2024-08-01', '2024-09-01', '2024-10-01', '2024-11-01',
               '2024-12-01', '2025-01-01'],
              dtype='datetime64[ns]', name='TIME_PERIOD', length=1662, freq=None)